# Smarte und interaktive Kurzskripte (smartiS)
P. Kersten, J. Jeschonowski, L.P. Kamegne-Kamdem, N.B. Neathery und D.M. Weis <br>

Hochschule Hamm-Lippstadt, Marker Allee 76–78, D-59063 Hamm
___

## 8. Das Rennen der Formen 

In [1]:
#____import____

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from smartiS import *
from ipywidgets import interact, HBox, VBox, Layout
from IPython.display import Markdown, clear_output, display, HTML
from matplotlib.pyplot import figure


javascript_funktion = {False: "hide()", True: "show()"}
schaltflaeche  = {False: "Quellcode anzeigen", True: "Quellcode verbergen"}


def quellcode_umschalten(state):

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_funktion[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))


def schaltflaeche_aktion(value):

    state = value.new
    quellcode_umschalten(state)
    value.owner.description = schaltflaeche[state]


state = False
quellcode_umschalten(state)

button = widgets.ToggleButton(state, description = schaltflaeche[state])
button.observe(schaltflaeche_aktion, "value")

display(button)

ToggleButton(value=False, description='Quellcode anzeigen')

### Beispiel: Formenrennen
Ein Hohlzylinder, ein Vollzylinder und eine massive Kugel mit den Massen M und den Radien R rollen an einer schiefen Ebene mit der Höhe h und einem Neigungswinkel von $\varphi$ um die wette. (Der Reibungsverlust sowie der Luftwiderstand sollen an dieser Stelle vernachlässigt werden)<br>
Wähle zwei Formen aus, die im Rennen gegeneinander antreten sollen und überprüfe, welche Form schlussendlich als Sieger hervorgeht!

**Knowledge Check(s):**
<ul>
    <li>Winkelgeschwindigkeit und Rotationsenergie</li>
    <li>Berechne verschiedene Trägheitsmomente und kontrolliere sie!</li>
    <li>Welche Form kommt am schnellsten unten an? Probiere es aus!</li>
    <li>Wie müssen die Massen verändert werden, damit alle Formen bei gleichbleibendem Radius genauso schnell sind? Und wie ist es vice versa (veränderbare Radien, gleiche Masse)</li>
</ul>


In [2]:
#Widgets
drop_down_1 = widgets.Dropdown(options=[('Kugel'),('Vollzylinder'), ('Hohlzylinder')],
                                     value='Kugel',
                                     description='Form 1:')

radius_1 = widgets.FloatSlider(min=0.05,
                               max=0.5,
                               step=0.01,
                               value=0.2,
                               description='Radius in m')

masse_1 = widgets.FloatSlider(min=0.1,
                              max=15.0,
                              step=0.1,
                              value=1.0,
                              description='Masse in kg')

radien = np.linspace(0, 0.5, 450, endpoint=True)
I = np.zeros(450)

#function, which defines moment of inertia for form
def parameters(form, radius, masse):
    
    if form == 'Kugel':
        I_ges = ((2/5) * radien**2 * masse)
        I = ((2/5) * radius**2 * masse)              #moment of inertia
        '''
        vel = np.sqrt((10/7)*9.81*(np.sin((angle/360)*2*np.pi) * length))   #velocity
        w = vel/radius                                            #angular velocity
        E_rot = 1/2 * I * w**2                               #energy of rotation
        '''
        fig, axs = plt.subplots(1)
        plt.suptitle('Trägheitmoment vs. Radius')         
        axs.plot(radien, I_ges, color='cyan', linestyle='dashed', linewidth=2)
        axs.plot(radius, I, 'ro')
        axs.axis([0, 0.51, 0, 4])
        axs.grid()
        axs.set_ylabel('Trägheitsmoment in kg/m²')
        axs.set_xlabel('Radius in m')
        
    elif form == 'Vollzylinder':
        I_ges = ((1/2) * radien**2 * masse)
        I = ((1/2) * radius**2 * masse)             #moment of inertia
        '''
        vel = np.sqrt((4/3)*9.81*(np.sin((angle/360)*2*np.pi) * length))   #velocity
        w = vz1_vel/radius                                                   #angular velocity
        E_rot = 1/2 * I * w**2                                   #energy of rotation
        '''
        fig, axs = plt.subplots(1)
        plt.suptitle('Trägheitmoment vs. Radius') 
        axs.plot(radien, I_ges, color='green', linestyle='dashed', linewidth=2)
        axs.plot(radius, I, 'ro')
        axs.axis([0, 0.51, 0, 4])
        axs.grid()
        axs.set_ylabel('Trägheitsmoment in kg/m²')
        axs.set_xlabel('Radius in m')
        
    else:
        I_ges = (radien**2 * masse)
        I = (radius**2 * masse)                     #moment of inertia
        '''
        vel = np.sqrt(9.81*(np.sin((angle/360)*2*np.pi) * length))   #velocity
        w = hz1_vel/radius                                             #angular velocity
        E_rot = 1/2 * I * w**2                                  #energy of rotation
        '''
        fig, axs = plt.subplots(1)
        plt.suptitle('Trägheitmoment vs. Radius') 
        axs.plot(radien, I_ges, color='red', linestyle='dashed', linewidth=2)
        axs.plot(radius, I, 'ro')
        axs.axis([0, 0.51, 0, 4])
        axs.plot()
        axs.grid()
        axs.set_ylabel('Trägheitsmoment in kg/m²')
        axs.set_xlabel('Radius in m')

#let widget interact with function and store value
widgets.interact(parameters, form = drop_down_1, radius = radius_1, masse = masse_1)

interactive(children=(Dropdown(description='Form 1:', options=('Kugel', 'Vollzylinder', 'Hohlzylinder'), value…

<function __main__.parameters(form, radius, masse)>

In [10]:
# animation for forms
class Race(smartiS):
    
    def __init__(self):
        
        # Ebenen definieren
        layers = ["bg",
                  "formen"
                 ]
        lx = 750
        ly = 600

        # smartiS-Klasse initialisieren
        super().__init__(canvases = layers,
                         width    = lx,
                         height   = ly)

        # Elemente erzeugen
       
        #Dropdown menu
        self.drop_down_1 = widgets.Dropdown(options=[('Kugel'),('Vollzylinder'), ('Hohlzylinder')],
                                     value='Kugel',
                                     description='Form 1:')
        self.drop_down_1.observe(self.draw_formen,"value")

        self.drop_down_2 = widgets.Dropdown(options=[('Kugel'), ('Vollzylinder'), ('Hohlzylinder')],
                                     value='Kugel',
                                     description='Form 2:')
        self.drop_down_2.observe(self.draw_formen,"value")

        #interactive sliders for radius and mass from form 1
        self.radius_1 = widgets.FloatSlider(min=0.05,
                                            max=0.5,
                                            step=0.01,
                                            value=0.25,
                                            description='Radius in m')
        self.radius_1.observe(self.draw_formen,"value")
        
        self.masse_1 = widgets.FloatSlider(min=0.05,
                                           max=15.0,
                                           step=0.1,
                                           value=1.0,
                                           description='Masse in kg')
        self.masse_1.observe(self.draw_formen,"value")

        self.radius_2 = widgets.FloatSlider(min=0.05,
                                            max=0.5,
                                            step=0.01,
                                            value=0.25,
                                            description='Radius in m')
        self.radius_2.observe(self.draw_formen,"value")
        
        self.masse_2 = widgets.FloatSlider(min=0.1,
                                           max=15.0,
                                           step=0.1,
                                           value=1.0,
                                           description='Masse in kg')
        self.masse_2.observe(self.draw_formen,"value")

        self.length = widgets.IntSlider(min=100,
                                            max=200,
                                            step=1,
                                            value=100,
                                            description='Länge der Rampe/Rennstrecke in cm')
        self.length.observe(self.draw_background,"value")
        
        self.angle = widgets.IntSlider(min=5,
                                       max=40,
                                       step=1,
                                       value=30,
                                       description='Steigungswinkel in °')
        self.angle.observe(self.draw_background,"value")
        
        self.play = widgets.Play(value=50,
                                 min=0,
                                 max=1200,
                                 step=1,
                                 interval=25,
                                 description="Starten",
                                 disabled=False)
        self.play.observe(self.draw_formen,"value")
        
        # Elemente darstellen
        display(HBox([
            self.canvas,
            VBox([HBox([], layout=Layout(height="600px")),
                  self.play,
                  self.drop_down_1,
                  self.masse_1,
                  self.radius_1,
                  self.drop_down_2,
                  self.masse_2,
                  self.radius_2,
                  self.length,
                  self.angle
                 ])
        ]))
        
        self.draw_background()
        self.draw_formen()
    
    
    
    def draw_background(self,*args):
        
        # Rampe zeichnen
        bg = self["bg"]
        length = self.length.value * 2
        angle = 180 - self.angle.value
        height = np.tan((-angle/180)*np.pi) * length
        
        with hold_canvas(bg):
            
            bg.begin_path()
            bg.clear()
            bg.move_to( 25, 575)
            bg.line_to(25, 575 - height)
            bg.stroke()
            bg.move_to( 25, 575)
            bg.line_to(length, 575)
            bg.stroke
            bg.move_to(25, 575 - height)
            bg.line_to(length, 575)
            bg.stroke()
            
            bg.begin_path()
            bg.move_to(25, 25)
            bg.line_to(25,225)
            bg.line_to(length, 225)
            bg.line_to(length,25)
            bg.line_to(25,25)
            bg.stroke()
    
    
    def draw_formen(self, *args):
        
        #Erste Form zeichnen
        f = self["formen"]
        t = self.play.value
        form1 = self.drop_down_1.value
        form2 = self.drop_down_2.value
        
        m1 = self.masse_1.value
        r1 = self.radius_1.value
        
        m2 = self.masse_2.value
        r2 = self.radius_2.value
        
        #Werte vom background bg
        length = self.length.value * 2
        angle = 180 - self.angle.value
        height = np.tan((-angle/180)*np.pi) * length
        
        #Form1
        if form1 == form1 or form2 == form2:
            f.clear()
        
        if form1 == 'Kugel':
            
            k1_x = 1/2 * (5/7 * 9.81 * np.sin((angle/360)*2*np.pi)) * t + 75
            
            with hold_canvas(f):

                f.stroke_arc(k1_x, 75, (r1*100), 0, 2*np.pi)
                f.fill_style = "#00ffff"
                f.fill_arc(k1_x, 75, r1*100, 0, 2*np.pi)
                f.fill_style = "#000000"
                f.fill_arc(k1_x, 75, 2, 0, 2*np.pi)
                
        elif form1 == 'Vollzylinder':

            vz1_x = 1/2 * (2/3 * 9.81 * np.sin((angle/360)*2*np.pi)) * t + 75
            
            with hold_canvas(f):

                
                f.stroke_rect(vz1_x-(r1*100), 75-(r1*100), 2 * r1*100, height=None)
                f.fill_style = "#00ff00"
                f.fill_rect(vz1_x-(r1*100), 75-(r1*100), 2 * r1*100, height=None)
                f.fill_style = "#000000"
                f.fill_arc(vz1_x, 75, 2, 0, 2*np.pi)
                
        elif form1 == 'Hohlzylinder':
            
            hz1_x = 1/2 * (1/2 * 9.81 * np.sin((angle/360)*2*np.pi)) * t + 75

            
            with hold_canvas(f):

                
                f.stroke_rect(hz1_x-(r1*100), 75-(r1*100), 2 * r1*100, height=None)
                f.fill_style = "#ff0000"
                f.fill_rect(hz1_x-(r1*100), 75-(r1*100), 2 * r1*100, height=None)
                f.fill_style = "#000000"
                f.fill_arc(hz1_x, 75, 2, 0, 2*np.pi)
                
            if hz1_x==length+25:
                self.play._playing = False
        
        #Form2
        if form2 == 'Kugel':
            
            k2_x = 1/2 * (5/7 * 9.81 * np.sin((angle/360)*2*np.pi)) * t + 75
            
            with hold_canvas(f):
                
                
                f.stroke_arc(k2_x, 175, r2*100, 0, 2*np.pi)
                f.fill_style = "#00ffff"
                f.fill_arc(k2_x, 175, r2*100, 0, 2*np.pi)
                f.fill_style = "#000000"
                f.fill_arc(k2_x, 175, 2, 0, 2*np.pi)
                
        elif form2 == 'Vollzylinder':
            
            vz2_x = 1/2 * (2/3 * 9.81 * np.sin((angle/360)*2*np.pi)) * t + 75
            
            with hold_canvas(f):

                
                f.stroke_rect(vz2_x-(r2*100), 175-(r2*100), 2 * r2*100, height=None)
                f.fill_style = "#00ff00"
                f.fill_rect(vz2_x-(r2*100), 175-(r2*100), 2 * r2*100, height=None)
                f.fill_style = "#000000"
                f.fill_arc(vz2_x, 175, 2, 0, 2*np.pi)
                
        elif form2 == 'Hohlzylinder':

            hz2_x = 1/2 * (1/2 * 9.81 * np.sin((angle/360)*2*np.pi)) * t + 75
            
            with hold_canvas(f):
                
                
                f.stroke_rect(hz2_x-(r2*100), 175-(r2*100), 2 * r2*100, height=None)
                f.fill_style = "#ff0000"
                f.fill_rect(hz2_x-(r2*100), 175-(r2*100), 2 * r2*100, height=None)
                f.fill_style = "#000000"
                f.fill_arc(hz2_x, 175, 2, 0, 2*np.pi)
        
            if hz2_x==length+25:
                self.play._playing = False
        

        
                
            
R = Race()

## Danksagung

Dieses digitale Lehr-/Lernmaterial wurde im Rahmen des Programms Fellowships für Innovationen in der digitalen Hochschullehre des Ministeriums für Kultur und Wissenschaft des Landes Nordrhein-Westfalen und des Stifterverbandes entwickelt. Es steht als Open Educational Ressources unter der Lizenz [Creative Commons Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](https://creativecommons.org/licenses/by-sa/4.0/) zur Verfügung.<br>
<img align="left\" src="https://licensebuttons.net/l/by-sa/3.0/88x31.png\">